In [298]:
import pandas as pd
import numpy as np
import re
import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression

In [299]:
# loading the Primary dataset:
nyt_lockdown = pd.read_csv("/Users/nitheeshvarma/Documents/NLP/Assignment 2 /NitheeshVarma_nlp_2_final/nyt_lockdowns.csv")
nyt_masking_distancing = pd.read_csv("/Users/nitheeshvarma/Documents/NLP/Assignment 2 /NitheeshVarma_nlp_2_final/nyt_masking_and_distancing.csv")
nyt_vaccination = pd.read_csv("/Users/nitheeshvarma/Documents/NLP/Assignment 2 /NitheeshVarma_nlp_2_final/nyt_vaccination.csv")

In [300]:
# Loading the Secondary dataset:
change_lockdown = pd.read_csv("/Users/nitheeshvarma/Documents/NLP/Assignment 2 /NitheeshVarma_nlp_2_final/change.org_lockdowns.csv")
change_masking_distancing = pd.read_csv("/Users/nitheeshvarma/Documents/NLP/Assignment 2 /NitheeshVarma_nlp_2_final/change.org_masking_and_distancing.csv")
change_vaccination = pd.read_csv("/Users/nitheeshvarma/Documents/NLP/Assignment 2 /NitheeshVarma_nlp_2_final/change.org_vaccination.csv")

In [301]:
# Data Partitioning
# Considering Vaccination among the three topics and doing the splitting.
x = nyt_vaccination["text"]
y = nyt_vaccination["label"]
x_train_nyt,x_test_nyt,y_train_nyt,y_test_nyt = train_test_split(x,y,test_size=0.3,random_state=45)#random_state is a seed value.
x_test_change = change_vaccination["text"]
y_test_change = change_vaccination["label"]

In [302]:
# Baseline Model Training
vectorizer = CountVectorizer()
x_train_nyt_bag = vectorizer.fit_transform(x_train_nyt)
x_test_nyt_bag = vectorizer.transform(x_test_nyt)
x_test_change_bag = vectorizer.transform(x_test_change)

In [303]:
model = LogisticRegression().fit(x_train_nyt_bag.toarray(),y_train_nyt)

In [304]:
model_test_nyt = model.predict(x_test_nyt_bag.toarray())
model_test_change = model.predict(x_test_change_bag.toarray())

In [305]:
#Model Evaluation 1
print(metrics.classification_report(y_test_nyt,model_test_nyt))

              precision    recall  f1-score   support

       False       0.88      0.88      0.88        97
        True       0.92      0.92      0.92       147

    accuracy                           0.90       244
   macro avg       0.90      0.90      0.90       244
weighted avg       0.90      0.90      0.90       244



In [306]:
print(metrics.classification_report(y_test_change,model_test_change))

              precision    recall  f1-score   support

       False       0.99      1.00      0.99      1473
        True       1.00      0.30      0.46        27

    accuracy                           0.99      1500
   macro avg       0.99      0.65      0.73      1500
weighted avg       0.99      0.99      0.98      1500



In [307]:
#The Baseline Model 1 overall acuuracy for Primary(nyt) and Secondary(change) are 0.90 and 0.99 respectively.

In [308]:
# Feature Engineering
def f1(m):
    return int(bool(re.search("unvaccinated",m))) #returns 1 if the word unvaccinated is present

In [309]:
def f2(n):
    return int(bool(re.search("powerful",n))) # returns 1 if the word powerful is present.

In [310]:
def f3(o):
    return int(bool(re.search("[A-Z][A-Z]+",o))) # returns 1 if all the words in the sentence are capital letters.

In [311]:
# Applying the feature 1 to x_train_nyt, x_test_nyt , x_test_change
f1_x_train_nyt = x_train_nyt.apply(f1)
f1_x_test_nyt  = x_test_nyt.apply(f1)
f1_x_test_change = x_test_change.apply(f1)

In [312]:
# Applying the feature 2 to x_train_nyt, x_test_nyt , x_test_change
f2_x_train_nyt = x_train_nyt.apply(f2)
f2_x_test_nyt  = x_test_nyt.apply(f2)
f2_x_test_change = x_test_change.apply(f2)

In [313]:
# Applying the feature 3 to x_train_nyt, x_test_nyt , x_test_change
f3_x_train_nyt = x_train_nyt.apply(f3)
f3_x_test_nyt  = x_test_nyt.apply(f3)
f3_x_test_change = x_test_change.apply(f3)

In [314]:
# Adding f1 to final 
nyt_train_final = np.insert(x_train_nyt_bag.toarray(),x_train_nyt_bag.shape[1],f1_x_train_nyt,axis=1)
nyt_test_final  = np.insert(x_test_nyt_bag.toarray(),x_test_nyt_bag.shape[1],f1_x_test_nyt,axis=1)
change_test_final = np.insert(x_test_change_bag.toarray(),x_test_change_bag.shape[1],f1_x_test_change,axis=1)

In [315]:
# Adding f2 to final
nyt_train_final = np.insert(x_train_nyt_bag.toarray(),x_train_nyt_bag.shape[1],f2_x_train_nyt,axis=1)
nyt_test_final  = np.insert(x_test_nyt_bag.toarray(),x_test_nyt_bag.shape[1],f2_x_test_nyt,axis=1)
change_test_final = np.insert(x_test_change_bag.toarray(),x_test_change_bag.shape[1],f2_x_test_change,axis=1)

In [316]:
# Adding f3 to final
nyt_train_final = np.insert(x_train_nyt_bag.toarray(),x_train_nyt_bag.shape[1],f3_x_train_nyt,axis=1)
nyt_test_final  = np.insert(x_test_nyt_bag.toarray(),x_test_nyt_bag.shape[1],f3_x_test_nyt,axis=1)
change_test_final = np.insert(x_test_change_bag.toarray(),x_test_change_bag.shape[1],f3_x_test_change,axis=1)

In [317]:
model_2 = LogisticRegression().fit(nyt_train_final,y_train_nyt)

In [318]:
model_2_test_nyt = model_2.predict(nyt_test_final)
model_2_test_change = model_2.predict(change_test_final)

In [319]:
#Model Evaluation 2:
print(metrics.classification_report(y_test_nyt,model_2_test_nyt))

              precision    recall  f1-score   support

       False       0.89      0.88      0.88        97
        True       0.92      0.93      0.92       147

    accuracy                           0.91       244
   macro avg       0.90      0.90      0.90       244
weighted avg       0.91      0.91      0.91       244



In [320]:
print(metrics.classification_report(y_test_change,model_2_test_change))

              precision    recall  f1-score   support

       False       0.99      1.00      0.99      1473
        True       1.00      0.33      0.50        27

    accuracy                           0.99      1500
   macro avg       0.99      0.67      0.75      1500
weighted avg       0.99      0.99      0.99      1500



In [321]:
# Reflection:
#The Baseline Model 2 overall acuuracy for Primary(nyt) and Secondary(change) are 0.91 and 0.99 respectively.
#For Primary(nyt) there was a slight increase in accuracy whereas for secondary the accuracy remains the same.